# 저수준 API 1.x를 그대로 사용 ( 2.x 기반 )

# 모듈가져오기

In [1]:
import tensorflow as tf
tf.__version__

'2.6.0'

# 2.x 기반에서 1.x가 구동되게 하는 조치사항

In [2]:
# 거의 코드상 변화가 없이 처리하는 방법
import tensorflow.compat.v1 as tf
# 2.x 버전의 행동들을 비활성해라
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# 데이터 준비

In [3]:
# 1. 여기서는 샘플 데이터가 MNIST(미리 제공되는) 이므로, 해당 패캐지가 변경어서 부득이 하게 수정

#from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.keras.datasets import mnist

import numpy as np

#mnist     = input_data.read_data_sets( './data/mnist/', one_hot=True )
# PIXEL     = mnist.train.images.shape[1]
# PIXEL_H   = int( np.sqrt(PIXEL) )
# PIXEL_W   = PIXEL_H
# LABEL_NUM = mnist.train.labels.shape[-1]

# (60000, 28, 28)
(X_train, y_train), (X_test, y_test) = mnist.load_data()
PIXEL_H = X_train.shape[1]
PIXEL_W = X_train.shape[2]
PIXEL   = PIXEL_H * PIXEL_W
LABEL_NUM = len(np.unique(y_train))
# 최대값 기준 정규화
X_train = X_train/np.max(X_train)
X_test  = X_test/np.max(X_test)
# one-hot 인코딩
y_train = tf.keras.utils.to_categorical( y_train,  LABEL_NUM) 
y_test  = tf.keras.utils.to_categorical( y_test,  LABEL_NUM) 

11501568/11490434 [==============================] - 0s 0us/step


# PART1


In [4]:
# part1 : Data Flow Graph 생성
x         = tf.placeholder( tf.float32, shape=(None, PIXEL), name='x' )

def make_FilterByWeight( name, shape ):  
  name = f'{name}_W'  
  tmp  = tf.truncated_normal(  shape, stddev=0.1 )  
  W    = tf.Variable( initial_value=tmp, name=name )
  return W;
def make_Bias( name, shape, value ):  
  name = f'{name}_b'  
  tmp  = tf.constant( value, shape=shape )  
  b    = tf.Variable( initial_value=tmp, name=name )
  return b
def make_conv2d( name, x, W ):  
  return tf.nn.conv2d(  x, filter=W, strides=[1,1,1,1], padding='SAME', name=name )

layer_name    = 'conv_1f'
with tf.name_scope( layer_name ) as scope:
  conv_1f_W   = make_FilterByWeight( layer_name, (5,5,1,32) )
  conv_1f_b   = make_Bias( layer_name, shape=(32,), value=0.1 )
  x_4d        = tf.reshape( x, ( -1, PIXEL_H, PIXEL_W, 1) )
  conv_1f     = make_conv2d( layer_name, x_4d,  conv_1f_W ) + conv_1f_b
  act_conv_1f = tf.nn.relu( conv_1f )

def make_MaxPooling( name, x ):
  name        = f'{name}_max'
  return tf.nn.max_pool(  x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name=name)  

layer_name    = 'pooling_1f'
with tf.name_scope( layer_name ) as scope:  
  pool_1f     = make_MaxPooling( layer_name, act_conv_1f )

layer_name     = 'conv_2f'
with tf.name_scope( layer_name ) as scope:  
  conv_2f_W    = make_FilterByWeight( layer_name, (5,5,32,32*2) )
  conv_2f_b    = make_Bias( layer_name, shape=(32*2,), value=0.1 )  
  conv_2f      = make_conv2d( layer_name, pool_1f,  conv_2f_W ) + conv_2f_b
  act_conv_2f  = tf.nn.relu( conv_2f )

layer_name     = 'pooling_2f'
with tf.name_scope( layer_name ) as scope:  
  pool_2f      = make_MaxPooling( layer_name, act_conv_2f )

layer_name = 'fc'
with tf.name_scope( layer_name ) as scope:  
  _, h, w, ch = pool_2f.shape
  in_channels = h * w * ch
  ch_size     = 1024
  tmp_x       = tf.reshape( pool_2f, ( -1,  in_channels) )
  fc_W        = make_FilterByWeight( layer_name, (in_channels, ch_size) )
  fc_b        = make_Bias( layer_name, (ch_size,), 0.1 )
  fc          = tf.matmul( tmp_x, fc_W ) + fc_b
  act_fc      = tf.nn.relu( fc )

layer_name        = 'act_fc_dropout'
with tf.name_scope( layer_name ) as scope:  
  keep_prob       = tf.placeholder( tf.float32 )
  act_fc_dropout  = tf.nn.dropout( act_fc, rate=1-keep_prob ) 

layer_name = 'output'
with tf.name_scope( layer_name ) as scope:  
  _, in_ch = act_fc_dropout.shape
  y_W      = make_FilterByWeight( layer_name, ( in_ch, LABEL_NUM )  )
  y_b      = make_Bias( layer_name,  (LABEL_NUM,), 0.1 )
  y_conv   = tf.matmul( act_fc_dropout, y_W ) + y_b
  y_conv   = tf.nn.softmax( y_conv )

y_ = tf.placeholder( tf.float32, shape=(None, LABEL_NUM), name='y_' )

layer_name      = 'loss'
with tf.name_scope( layer_name ) as scope:
  cross_entropy = -tf.reduce_sum( y_ * tf.log( y_conv )  )

layer_name = 'adam'
with tf.name_scope( layer_name ) as scope:  
  optimzer = tf.train.AdamOptimizer()  
  train    = optimzer.minimize( cross_entropy )

layer_name = 'predict'
with tf.name_scope( layer_name ) as scope:
  predict  = tf.equal( tf.argmax( y_conv, 1 ) , tf.argmax( y_, 1 ) )
  accuracy = tf.reduce_mean( tf.cast( predict, tf.float32 ) )

In [5]:
# part2 : 실제 학습/평가 수행
def make_Feed_Dict( x_data, labels, prob ):    
  return {
      x:x_data,
      y_:labels,
      keep_prob:prob
  }

#with tf.device('/device:GPU:0'):
with tf.Session() as sess:
  TRAIN_COUNT  = 3000
  BATCH_SIZE   = 50    
  VERBOSE_TERM = 100
  sess.run( tf.global_variables_initializer() )  
  #test_fd = make_Feed_Dict( mnist.test.images,  mnist.test.labels, 1.0 )
  test_fd = make_Feed_Dict( X_test.reshape(-1, PIXEL),  y_test, 1.0 )
  for step in range( TRAIN_COUNT ):
    # 훈련용 데이터 교체 -> next_batch은 셔플해서 랜덤하게 BATCH_SIZE 만큼 획득
    #batch = mnist.train.next_batch( BATCH_SIZE )            
    # 그냥 앞에서부터 차례대로 뽑겟다 -> 데이터의 범위를 넘어설수도 있기 때문에 주의->훈련성과는 포기
    # 돌아가는것 확인하면 중단시키겟다
    start_idx = step*BATCH_SIZE
    end_idx   = (step+1)*BATCH_SIZE
    batch     = [ X_train[ start_idx: end_idx].reshape(-1, PIXEL), y_train[start_idx : end_idx] ]

    train_fd = make_Feed_Dict( batch[0], batch[1], 0.1 )
    acc, _, loss_log = sess.run( [accuracy, train, cross_entropy] , feed_dict=train_fd )
    #loss_logs.append( loss_log )
    if step % VERBOSE_TERM == 0:      
      acc = sess.run( accuracy, feed_dict=test_fd )
      print( f'step={step:4} acc={acc:20}  loss={loss_log:20}' )          
    
  acc = sess.run( accuracy, feed_dict=test_fd )
  print( f'step={step:4} acc={acc:20}  loss={loss_log:20}' )


step=   0 acc= 0.15289999544620514  loss=   1058.667724609375
step= 100 acc=  0.8952000141143799  loss=   51.69630813598633
step= 200 acc=  0.9387000203132629  loss=    44.1787223815918
step= 300 acc=  0.9514999985694885  loss=   21.61581802368164


KeyboardInterrupt: ignored